# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications.

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below.

<img src='https://github.com/SOG-web/udacity_image_classifier_project/blob/main/assets/Flowers.png?raw=1' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [5]:
import torch
import numpy as np
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import json
from collections import OrderedDict
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import urllib.request
import tarfile

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.


In [6]:
# Downloading and extracting the flower dataset
def download_and_extract_data(url, destination_folder):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    filename = url.split('/')[-1]
    urllib.request.urlretrieve(url, os.path.join(destination_folder, filename))

    with tarfile.open(os.path.join(destination_folder, filename), 'r:gz') as tar:
        tar.extractall(destination_folder)

# Setting up directories for training, validation, and testing
data_dir = 'flowers'
train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')


download_and_extract_data('https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz', 'flowers')

In [7]:
# Define transformations for training, validation, and testing sets
def get_data_transforms():
    return {
        'train': transforms.Compose([
            transforms.RandomRotation(30),
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'valid': transforms.Compose([
            transforms.Resize(255),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize(255),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }


# Loading datasets using ImageFolder
def load_datasets(data_dir, transforms):
    from torchvision import datasets
    return {
        'train': datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transforms['train']),
        'valid': datasets.ImageFolder(root=os.path.join(data_dir, 'valid'), transform=transforms['valid']),
        'test': datasets.ImageFolder(root=os.path.join(data_dir, 'test'), transform=transforms['test'])
    }


# Creating data loaders
def create_data_loaders(datasets):
    return {
        'train': torch.utils.data.DataLoader(datasets['train'], batch_size=64, shuffle=True),
        'valid': torch.utils.data.DataLoader(datasets['valid'], batch_size=64),
        'test': torch.utils.data.DataLoader(datasets['test'], batch_size=64)
    }


transforms = get_data_transforms()
datasets = load_datasets(data_dir, transforms)
data_loaders = create_data_loaders(datasets)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [10]:
# Defining the mapping from category label to category name
def get_category_mapping(json_file):
    with open(json_file, 'r') as f:
        return json.load(f)


cat_to_name = get_category_mapping('cat_to_name.json')
# prints json file
print(cat_to_name)

no_output_categories = len(cat_to_name)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# Building and training the network

hidden_units = 4096
model = models.vgg16_bn(pretrained=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def build_and_train_model():

    for param in model.parameters():
        param.requires_grad = False

    model.classifier = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(25088, hidden_units)),
        ('relu', nn.ReLU()),
        ('dropout1', nn.Dropout(0.05)),
        ('fc2', nn.Linear(hidden_units, no_output_categories)),
        ('output', nn.LogSoftmax(dim=1))
    ]))

    model.to(device)
    print(f'The device in use is {device}.\n')

    epochs = 10
    optimizer = optim.Adam(model.classifier.parameters(), lr=.001)
    criterion = nn.NLLLoss()

    print_every = 20
    running_loss = running_accuracy = 0
    validation_losses, training_losses = [],[]

    for e in range(epochs):
        batches = 0

        model.train()

        for images, labels in data_loaders['train']:
            start = time.time()
            batches += 1

            images, labels = images.to(device), labels.to(device)

            log_ps = model.forward(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()

            ps = torch.exp(log_ps)
            top_ps, top_class = ps.topk(1, dim=1)
            matches = (top_class == labels.view(*top_class.shape)).type(torch.FloatTensor)
            accuracy = matches.mean()

            optimizer.zero_grad()
            running_loss += loss.item()
            running_accuracy += accuracy.item()

            if batches % print_every == 0:
                end = time.time()
                training_time = end - start
                start = time.time()

                validation_loss = 0
                validation_accuracy = 0

                model.eval()
                with torch.no_grad():
                    for images, labels in data_loaders['valid']:
                        images, labels = images.to(device), labels.to(device)
                        log_ps = model.forward(images)
                        loss = criterion(log_ps, labels)
                        ps = torch.exp(log_ps)
                        top_ps, top_class = ps.topk(1, dim=1)
                        matches = (top_class == labels.view(*top_class.shape)).type(torch.FloatTensor)
                        accuracy = matches.mean()

                        validation_loss += loss.item()
                        validation_accuracy += accuracy.item()

                end = time.time()
                validation_time = end - start
                validation_losses.append(running_loss / print_every)
                training_losses.append(validation_loss / len(data_loaders['valid']))

                print(f'Epoch {e+1}/{epochs} | Batch {batches}')
                print(f'Running Training Loss: {running_loss/print_every:.3f}')
                print(f'Running Training Accuracy: {running_accuracy/print_every*100:.2f}%')
                print(f'Validation Loss: {validation_loss/len(data_loaders["valid"]):.3f}')
                print(f'Validation Accuracy: {validation_accuracy/len(data_loaders["valid"])*100:.2f}%')

                running_loss = running_accuracy = 0
                model.train()


build_and_train_model()

The device in use is cpu.

Epoch 1/10 | Batch 20
Running Training Loss: 4.815
Running Training Accuracy: 11.80%
Validation Loss: 3.243
Validation Accuracy: 30.66%


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# Validating on the test set
def validate_on_test_set():
    import torch
    start_time = time.time()
    print('Validation started.')

    test_accuracy = 0

    for images, labels in data_loaders['test']:
        model.eval()
        images, labels = images.to(device), labels.to(device)
        log_ps = model.forward(images)
        ps = torch.exp(log_ps)
        top_ps, top_class = ps.topk(1, dim=1)
        matches = (top_class == labels.view(*top_class.shape)).type(torch.FloatTensor)
        accuracy = matches.mean()
        test_accuracy += accuracy

    end_time = time.time()
    print('Validation ended.')
    validation_time = end_time - start_time
    print('Validation time: {:.0f}m {:.0f}s'.format(validation_time / 60, validation_time % 60))

    print(f'Test Accuracy: {test_accuracy/len(data_loaders["test"])*100:.2f}%')


validate_on_test_set()

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# Saving the checkpoint
def save_checkpoint(model, filepath, class_to_idx):
    checkpoint = {
        'model_arch': 'vgg16_bn',
        'clf_input': 25088,
        'clf_output': len(class_to_idx),
        'clf_hidden': 4096,
        'state_dict': model.state_dict(),
        'class_to_idx': class_to_idx
    }

    torch.save(checkpoint, filepath)


save_checkpoint(model, 'flower_classifier_checkpoint.pth', datasets['train'].class_to_idx)

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# Loading the checkpoint and rebuilding the model
def load_checkpoint(filepath, device):
    import torch

    checkpoint = torch.load(filepath, map_location=device)

    return checkpoint['model_arch'], checkpoint['clf_input'], checkpoint['clf_output'], checkpoint['clf_hidden'], \
           checkpoint['state_dict'], checkpoint['class_to_idx']

model_arch, input_units, output_units, hidden_units, state_dict, class_to_idx = load_checkpoint('flower_classifier_checkpoint.pth', device)
model.load_state_dict(state_dict)


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network.

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training.

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation.

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
# Preprocessing an image for prediction
def process_image(image_path):
    from PIL import Image
    import numpy as np
    import torch
    from torchvision import transforms

    image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = preprocess(image).unsqueeze(0)
    return image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
        plt.tick_params(
            axis='both',
            which='both',
            bottom=False,
            top=False,
            left=False,
            labelbottom=False,
            labelleft=False,)

    # We need to move the color channel from the first dimension to the third dimension.
    # PyTorch expects color to be in the 1st dim, but PIL expects it to be in the 3rd!
    image = image.transpose((1, 2, 0))

    # undoes preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # image needs to be clipped between 0 and 1
    # otherwise it looks like noise when displayed
    image = np.clip(image, 0, 1)

    ax.imshow(image)

    return ax

In [ ]:
practice_img = './flowers/test/70/image_06186.jpg'

# displays original image
processed_image = Image.open(practice_img)
processed_image


# displays preprocessed image
imshow(process_image(practice_img))

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
# Making predictions on an image
def predict(image_path, model, idx_to_class, topk=5):
    image = process_image(image_path)
    model.eval()
    outputs = model(image)
    ps = torch.exp(outputs)
    top_ps, top_class = ps.topk(topk, dim=1)
    top_ps = top_ps.squeeze().tolist()
    top_class = [idx_to_class[idx] for idx in top_class.squeeze().tolist()]
    return top_ps, top_class


image_path = './flowers/test/70/image_05331.jpg'
top_ps, top_class = predict(image_path, model, {v: k for k, v in datasets['train'].class_to_idx.items()})

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='https://github.com/SOG-web/udacity_image_classifier_project/blob/main/assets/inference_example.png?raw=1' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# Displaying predictions
def display_prediction(image_path, probs, classes, cat_to_name):
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg

    fig, (ax1, ax2) = plt.subplots(figsize=(6, 9), nrows=2)
    img = mpimg.imread(image_path)
    ax1.imshow(img)
    ax1.axis('off')

    ax2.barh(np.arange(len(classes)), probs)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(len(classes)))
    ax2.set_yticklabels([cat_to_name[class_idx] for class_idx in classes], size='medium');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

display_prediction(image_path, top_ps, top_class, cat_to_name)